In [1]:
import pandas as pd
from sklearn import model_selection, preprocessing, metrics, svm
import statsmodels.api as sm
from autofeat import AutoFeatRegressor, FeatureSelector
import seaborn as sns
import numpy as np

# Read Data

In [2]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_train.describe()

,Id,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4999.50000,1.576701,-11.299161,3.907769,0.450645,0.871660,-1.579386,-2.087579,1.917499,1.308422,4.861379,17.415757
std,2886.89568,310.616925,999.012531,1000.586380,1003.073868,999.249352,992.302193,997.872541,1011.010545,987.168706,991.729300,991.443805
min,0.00000,-1406.400147,-4659.952967,-3979.924822,-3740.100638,-3666.661811,-3532.991929,-3694.285292,-4852.117653,-3745.535589,-4446.632241,-3581.045869
25%,2499.75000,-207.214300,-691.133553,-664.978917,-684.337536,-678.238599,-667.807938,-679.852326,-681.952398,-646.896660,-666.371096,-628.837251
50%,4999.50000,5.252991,-13.957911,15.803377,-3.275047,0.500737,-1.850882,-6.014849,-13.882376,2.266320,13.982412,17.454569
75%,7499.25000,210.453130,657.367062,673.309887,677.438589,676.898037,660.881459,669.623894,692.497137,668.526779,672.569701,684.593863
max,9999.00000,1228.193658,3844.824744,3852.020172,4019.773551,4241.771913,3803.844039,3687.019024,3483.755269,3491.550364,3831.790162,3766.942168


In [19]:
df_train, df_val = model_selection.train_test_split(df_train, test_size=0.2)
X_train = df_train.drop(columns=['y', 'Id'])
y_train = df_train.y
X_val = df_val.drop(columns=['y', 'Id'])
y_val = df_val.y

X_test = df_test.drop(columns=['Id'])
X_train.size, X_val.size

(10720, 2680, numpy.float64)

# Data Scaling

In [4]:
scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_val = scaler.transform(X_val)
#X_test = scaler.transform(X_test)

MinMaxScaler(feature_range=(-1, 1))

# Fit regression

In [5]:
models = {}

# GLM
#reg = sm.GLM(y_train, X_train, sm.families.Gaussian()).fit()
#reg.summary()
#models['glm'] = reg

In [6]:
# SVM
reg = svm.SVR()
reg.fit(X_train, y_train)
models['svm'] = reg

In [7]:
# autofeat
reg = AutoFeatRegressor()
df = reg.fit_transform(X_train, y_train)
models['autofeat'] = reg

/Users/jodok/02 Code/spring21-JodokVieli/venv/lib/python3.8/site-packages/numpy/core/_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/Users/jodok/02 Code/spring21-JodokVieli/venv/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)
/Users/jodok/02 Code/spring21-JodokVieli/venv/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)


In [34]:
# mean
class MeanModel:
    def fit(self, x, y):
        print("no need to fit data.")
    def predict(self, x):
        return np.mean(x, axis=1)
    
model = MeanModel()
model.fit(X_train, y_train)
models['manual'] = model

no need to fit data.


# Evaluate

In [35]:
for name, model in models.items():
    y_pred = model.predict(X_val)
    rmse = metrics.mean_squared_error(y_val, y_pred)**0.5
    print(f'{name}: {rmse}')

svm: 160.5599239216804
autofeat: 1.389255918575364e-13
manual: 6.034996195686455e-14


# Predict

In [33]:
reg = models['manual']
y_res = reg.predict(X_test)
df_res = pd.DataFrame({'Id': df_test.Id, 'y': y_res})
df_res.to_csv('results.csv', header=True, index=False)